In [1]:
# This is code using the Paci 2013 model and the genetic algorithm
# This code will fit experimental data of iPSC-CM
# Written by Amy Gutierrez

In [49]:
#importing libraries

import math
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from geneticalgorithm import geneticalgorithm as ga

In [50]:
# Fitness Function

celltype = 2
atrial = pd.read_csv("Atrial_Day46to49.csv", delimiter=",")
atrialVolt = atrial['atrialVolt']
atrialVolt = np.array(atrialVolt)
atrialVolt = atrialVolt + 30

initmeans = [3.4e3,29.1655987640683,
30.2556975158104,2.5434839512778,26.0345343708055,9.0416289e-5,4500,1.8,
0.4,
0.82918655616393,0.71903515994148,
1.0447445959728,0.5984574443992,4e-4,30,]

names = ['GNa', 'GK1','GKr','GKs','Gto','GCaL','KNaCA',
         'PNaK','GpCa','GbNa','GbCa','G_RyR','Vmaxup','Vleak','Gf']

nvars = len(names)
variation = 1

LB = np.zeros((1,nvars))
popsize = 1000
initpop = abs(2*variation*np.random.rand(popsize, nvars) - variation) * initmeans

In [51]:
# Paci13 dydt

def dydt_paci13_temp(t,statevar,Istim,globalVars):
    
    Q10_NaK = globalVars[0]
    Q10_KmNai = globalVars[1]
    Q10_CaL = globalVars[2]
    Q10_NCX = globalVars[3]
    Q10_SLCaP = globalVars[4]
    Q10_SRCaP = globalVars[5]
    Q10_f = globalVars[6]
    Q10_K1 = globalVars[7]
    Q10_Na = globalVars[8]
    Q10_to = globalVars[9]
    Q10_Kr = globalVars[10]
    Q10_Ks = globalVars[11]
    Q10_bNa = globalVars[12]
    Q10_bCa = globalVars[13]
    Q10_leak = globalVars[14]
    Q10_rel = globalVars[15]
    Nao = globalVars[16]
    Ko = globalVars[17]
    Cao= globalVars[18]
    Ki = globalVars[19]
    Cm = globalVars[20]
    Vc = globalVars[21]
    Vsr = globalVars[22]
    arel = globalVars[23]
    brel = globalVars[24]
    crel = globalVars[25]
    Bufc = globalVars[26]
    Bufsr = globalVars[27]
    Kbufc = globalVars[28] 
    Kbufsr = globalVars[29]
    Kup = globalVars[30]
    KpCa = globalVars[31]
    F = globalVars[32]
    R = globalVars[33]
    T = globalVars[34]
    L0 = globalVars[35]
    Pkna = globalVars[36]
    Ksat = globalVars[37]
    KmCa = globalVars[38]
    KmNai = globalVars[39]
    alpha = globalVars[40]
    gamma = globalVars[41]
    KmNa = globalVars[42]
    Kmk = globalVars[43]
    GNa = globalVars[44]
    GK1 = globalVars[45]
    GKr = globalVars[46]
    GKs = globalVars[47]
    Gto = globalVars[48]
    GCaL = globalVars[49]
    PNaK = globalVars[50]
    KNaCA = globalVars[51]
    GpCa = globalVars[52]
    GbNa = globalVars[53]
    GbCa = globalVars[54]
    G_RyR = globalVars[55]
    Vmaxup = globalVars[56]
    Vleak = globalVars[57]
    Gf = globalVars[58]
    celltype = globalVars[59]
    
    
    # reversal potentials
    ENa = ((R*T)/(F))*(log(Nao/statevar[15])) 
    Ek = ((R*T)/(F))*(log(Ko/Ki))
    Eks = ((R*T)/(F))*log((Ko+(Pkna*Nao))/(Ki+(Pkna*statevar[15])))
    ECa = ((0.5*R*T)/(F))*log(Cao/statevar[16])
    Ef = -0.017 
                              
                              
    # Temp scales for time constants                  
    QCaL = Q10_CaL**((310-T)/10)
    Qf = Q10_f**((310-T)/10)
    QNa = Q10_Na**((310-T)/10)
    Qto = Q10_to**((310-T)/10)
    QKr = Q10_Kr**((310-T)/10)
    QKs = Q10_Ks**((310-T)/10) 
    QNaK = Q10_NaK**((310-T)/10)
    QKmNai = Q10_KmNai**((310-T)/10)
    Qrel = Q10_rel**((310-T)/10)
    QK1 = Q10_K1**((T-310)/10)
    QNCX = Q10_NCX**((T-310)/10)
    QSLCaP = Q10_SLCaP**((T-310)/10)
    QSRCaP = Q10_SRCaP**((T-310)/10)
    QbNa = Q10_bNa**((310-T)/10)
    QbCa = Q10_bCa**((310-T)/10)
    Qleak = Q10_leak**((310-T)/10)   

                              
    # Na Current, INa
    INa = GNa*(statevar[2]^3)*statevar[0]*statevar[1]*(statevar[14]-ENa)                           
    hinf = (1+exp((statevar[14]*1000+72.1)/5.7))**(-1/2) 

    if (statevar[14] < -0.040):
        alphah = 0.057*exp(-(statevar[14]*1000.0+80.0)/6.8) 
        betah = 2.7*exp(0.079*statevar[14]*1000.0)+3.1*(10.0**5.0)*math.exp(0.3485*statevar[14]*1000.0)
        taoh = QNa*(1.5/((alphah+betah)*1000.0)) 
    else:
        alphah = 0 
        betah = 0.77/(0.13*(1.0+math.exp((statevar[14]*1000.0+10.66)/-11.1)))
        taoh  = QNa*(2.542/1000)
    dhdt = (hinf - statevar[0])/taoh
    
    
    # INa, j gate
    jinf = 1.0/math.sqrt(1.0+exp((statevar[14]*1000.0+72.1)/5.7))
    
    if (statevar[14] < -40e-3): 
        alphaj = (-25428.0*math.exp(0.2444*statevar[14]*1000.0)-6.948*(10.0**-6.0)*math.exp(-0.04391*statevar[14]*1000.0))*(statevar[14]*1000.0+37.78)/(1.0+math.exp(0.311*(statevar[14]*1000.0+79.23)))
        betaj = 0.02424*math.exp(-0.01052*statevar[14]*1000.0)/(1.0+math.exp(-0.1378*(statevar[14]*1000.0+40.14)))
    else:
        alphaj = 0
        betaj = 0.6*math.exp(0.057*statevar[14]*1000.0)/(1.0+math.exp(-0.1*(statevar[14]*1000.0+32.0)))
    
    taoj = QNa*(7.0/((alphaj+betaj)*1000.0))
    djdt = (jinf - statevar[1])/taoj
           
        
    # INa, m gate
    minf = 1.0/(1.0+math.exp((-statevar[14]*1000.0-34.1)/5.9))**(1.0/3.0)
    alpham = 1.0/(1.0+math.exp((-statevar[14]*1000.0-60.0)/5.0))
    betam = 0.1/(1.0+math.exp((statevar[14]*1000.0+35.0)/5.0))+0.1/(1.0+math.exp((statevar[14]*1000.0-50.0)/200.0))
    taum = QNa*(1.0*alpham*betam/1000.0)
    dmdt = (minf-statevar[2])/taum
    
    
    # L-type Calcium current
    ICaL = GCaL*4.0*statevar[14]*(F**2.0)/(R*T)*(statevar[16]*math.exp(2.0*statevar[14]*F/(R*T))-0.341*Cao)/(math.exp(2.0*statevar[14]*F/(R*T))-1.0)*statevar[3]*statevar[4]*statevar[6]*statevar[6]
    
    # ICaL, d gate
    if (celltype == 1):
        dinf = 1.0/(1.0+exp(-(statevar[14]*1000.0+9.1)/7.0)) 
    else: 
        dinf = (1+exp((-(statevar[14]*1000+5.986))/7))**(-1)
        
    alphad = 0.25+1.4/(1.0+math.exp((-statevar[14]*1000.0-35.0)/13.0))
    betad = 1.4/(1.0+math.exp((statevar[14]*1000.0+5.0)/5.0))
    gammad = 1.0/(1.0+math.exp((-statevar[14]*1000.0+50.0)/20.0))
    taod = QCaL*((alphad*betad+gammad)*1.0/1000.0)
    dddt = ((dinf - statevar[3]) / taod)
    
    # ICaL, fCa gate
    alphafCa = 1.0/(1.0+(statevar[16]/0.0006)**8.0)
    betafCa = 0.1/(1.0+math.exp((statevar[16]-0.0009)/0.0001))
    gammafCa = 0.3/(1.0+math.exp((statevar[16]-0.00075)/0.0008))
    fCainf = (alphafCa+betafCa+gammafCa)/1.3156
    taofCa = QCaL*(2/1000)
    if ((fCainf > statevar[4]) and (statevar[14] > -0.060)): 
        dfCadt = 0 
    else:
        dfCadt = ((fCainf - statevar[4]) / taofCa)
    
    # ICaL, f1 gate
    if (celltype == 1):
        f1inf = 1.0/(1.0+math.exp((statevar[14]*1000.0+26.0)/3.0))
    else:
        f1inf = (1+exp((statevar[14]*1000+25.226)/3))**(-1)

    if (f1inf - statevar[5] > 0.0):
        taof1constant = QCaL*(1.0 + 1433.0*(statevar[16]-(50e-6)))
    else:
        taof1constant = QCaL*1.0
        
    taof1 = QCaL*(((20.0+1102.5*math.exp(-((statevar[14]*1000.0+27.0)**2.0/15.0)**2.0)+200.0/(1.0+math.exp((13.0-statevar[14]*1000.0)/10.0))+180.0/(1.0+math.exp((30.0+statevar[14]*1000.0)/10.0)))*taof1constant/1000.0))
    df1dt = ((f1inf - statevar[5]) / taof1)
    
    # ICaL, f2 gate
    if (celltype==1):
        f2inf = (0.67/(1+math.exp((statevar[14]*1000+35)/4)))+0.33 
    else:
        f2inf = (0.67/(1+math.exp((statevar[14]*1000+31.226)/4)))+0.33

    if (celltype==1):
        taof2 = ((600*math.exp(-(((statevar[14]*1000+25)**2)/170))) + (31/(1+math.exp((-statevar[14]*1000+25)/10))) +  (16/(1+math.exp((statevar[14]*1000+30)/10))))/1000
    else:
        taof2 = (((600*math.exp(-(((statevar[14]*1000+25)**2)/170))) + (31/(1+math.exp((-statevar[14]*1000+25)/10))) +  (16/(1+math.exp((statevar[14]*1000+30)/10))))*2)/1000

    df2dt = ((f2inf - statevar[6]) / taof2)
    
    
    # Transient outward current, Ito
    Ito = (Gto*statevar[7]*statevar[8]*(statevar[14] - Ek))
    
    # Ito, r gate
    rinf = 1.0/(1.0+math.exp(-(statevar[14]*1000.0-22.3)/18.75))
    taor = Qto*((2.75352+14.40516/(1.037*math.exp(0.09*(statevar[14]*1000.0+30.61))+0.369*math.exp(-0.12*(statevar[14]*1000.0+23.84))))/1000.0)
    drdt = (rinf - statevar[7]/(taor))
    
    # Ito, q gate
    qinf = (1+math.exp((statevar[14]*1000+53)/13))**(-1)
    taoq = Qto*(((39.102 / ((0.57*math.exp(-0.08*(statevar[14]*1000+44))) + (0.065*math.exp(0.01*(statevar[14]*1000+45.93))))) + 6.06)/1000)
    dqdt = ((qinf - statevar[8]) / taoq)     
        


    # Rapid delayed rectifier K+ current, IKr
    IKr = GKr*((Ko/5.4)**.5)*statevar[9]*statevar[10]*(statevar[14]-Ek)
    
    # IKr, Xr1 gate
    Vonehalf = 1000.0*(-R*T/(F*2.3)*log((1.0+Cao/2.6)**4.0/(L0*(1.0+Cao/0.58)**4.0))-0.019)
    xr1inf = 1.0/(1.0+math.exp((Vonehalf-statevar[14]*1000.0)/4.9))
    alphaxr1 = 450.0/(1.0+math.exp((-45.0-statevar[14]*1000.0)/10.0))
    betaxr1 = 6.0/(1.0+math.exp((30.0+statevar[14]*1000.0)/11.5))
    taoxr1 = QKr*(1.0*alphaxr1*betaxr1/1000.0)
    dXr1dt = ((xr1inf - statevar[9]) / taoxr1)
            
    
    # IKr, Xr2 gate
    xr2inf = 1.0/(1.0+math.exp((statevar[14]*1000.0+88.0)/50.0))
    alphaxr2 = 3.0/(1.0+math.exp((-60.0-statevar[14]*1000.0)/20.0))
    betaxr2 = 1.12/(1.0+math.exp((-60.0+statevar[14]*1000.0)/20.0))
    taoxr2 = QKr*(1.0*alphaxr2*betaxr2/1000.0)
    dXr2dt = ((xr2inf-statevar[10]) / (taoxr2)) 
            

    # Slow delayed rectifier K+ current, IKs
    IKs = GKs*(statevar[11]**2)*(1+(0.6/(1+(((3.8e-5)/statevar[16])**1.4))))*(statevar[14]-Eks)
    
    # IKs, Xs gate
    XSinf = (1+ math.exp((-20-statevar[14]*1000)/16))**(-1)
    alphaxs = 1100/((1+math.exp((-10-statevar[14]*1000)/6))**0.5) 
    betaxs = (1+math.exp((-60+statevar[14]*1000)/20))**(-1)  
    taoxs = QKs*((alphaxs*betaxs)/1000)
    dXsdt = ((XSinf - statevar[11]) / taoxs)
            
            
    # Inward rectifier K+ current, IK1
    alphaK1 = 3.91/(1.0+math.exp(0.5942*(statevar[14]*1000.0-Ek*1000.0-200.0)))
    alphaK1 = alphaK1 * QK1
    betaK1 = (-1.509*math.exp(0.0002*(statevar[14]*1000.0-Ek*1000.0+100.0))+math.exp(0.5886*(statevar[14]*1000.0-Ek*1000.0-10.0)))/(1.0+math.exp(0.4547*(statevar[14]*1000.0-Ek*1000.0)))
    betaK1 = betaK1 * QK1 
    XK1inf = alphaK1/(alphaK1+betaK1)
    IK1 = GK1*XK1inf*(statevar[14]-Ek)*math.sqrt(Ko/5.4)
            
            
            
    # Hyperpolarization activated funny current, If
    If = GF * statevar[12]*(statevar[14]-Ef)
            
    # If, Xf gate
    xfinf = (1+math.exp((statevar[14]*1000+77.85)/5))**(-1)
    taof = Qf * (1900/(1+math.exp((statevar[14]*1000+15)/10)))
    dXfdt = ((xfinf - statevar[12]) / taof)

             
             
    # Na+K+ pump current, INaK
    INaK = PNaK*Ko/(Ko+(QNaK*Kmk))*statevar[15]/(statevar[15]+(QKmNai*KmNa))/(1.0+0.1245*math.exp(-0.1*statevar[14]*F/(R*T))+0.0353*math.exp(-statevar[14]*F/(R*T)))
    
             
    # Na+/Ca2+ exchanger current, INaCa
    INaCa = QNCX*KNaCA*(math.exp(gamma*statevar[14]*F/(R*T))*(statevar[15]**3.0)*Cao-math.exp((gamma-1.0)*statevar[14]*F/(R*T))*(Nao**3.0)*statevar[16]*alpha)/((KmNai**3.0+Nao**3.0)*(KmCa+Cao)*(1.0+Ksat*math.exp((gamma-1.0)*statevar[14]*F/(R*T))))
             
    if (celltype==1):
        Irel = G_RyR* ((crel+arel*(statevar[17]**2.0)/(brel**2.0+statevar[17]**2.0))*statevar[3]*statevar[13]*0.0411 )
    else:
        Irel = G_RyR* ((((arel*(statevar[17]**2))/((brel**2)+(statevar[17]**2)))+crel)*statevar[3]*statevar[13]*0.0556)
  

    # Other Calcium Currents
    Iup = QSRCaP*Vmaxup/(1.0+Kup**2.0/statevar[16]**2.0)
    Ileak = (statevar[17]-statevar[16])*Vleak
             
    if (statevar[16] <= 0.00035):
        ginf = 1.0/(1.0+(statevar[16]/0.00035)**6.0)
    else:
        ginf = 1.0/(1.0+(statevar[16]/0.00035)**16.0)
                
    taog = Qrel*0.002 
    
    if ((ginf > statevar[13]) and (statevar[14] > -0.060)):
        dgdt = 0 
    else:
        dgdt = ((ginf - statevar[13])/taog) 
 
    Caibufc = 1.0/(1.0+Bufc*Kbufc/(statevar[16]+Kbufc)**2.0)
    Casrbufsr = 1.0/(1.0+Bufsr*Kbufsr/(statevar[17]+Kbufsr)**2.0)          
             
    IPCa = QSLCaP*(GpCa*statevar[16])/(statevar[16]+KpCa)         
    IbCa = GbCa*(statevar[14]-ECa)         
    dCaidt = Caibufc*(Ileak-Iup+Irel-(((ICaL+IbCa+IPCa-(2*INaCa))/(2*Vc*F*1e-18))*Cm)) 
    dCasrdt = Casrbufsr*Vc/Vsr*(Iup-(Irel+Ileak))      
    
    
    # Background Currents
    IbNa = GbNa*(statevar[14]-ENa) 
    
    # Sodium Dynamics
    dNaidt = -Cm*((INa+IbNa+(3*INaK)+(3*INaCa))/(F*Vc*1e-18)) 
    
    
    # SOLVE DERIVATIVE
    Iion = (IK1 + Ito + IKr + IKs + ICaL + INaK + INa + INaCa + IPCa + If + IbNa + IbCa)
    dVdt = - (Iion + Istim)
    currents = [IK1, Ito, IKr, IKs, ICaL, INaK, INa, INaCa, IPCa, If, IbNa, IbCa]
    
    
    deriv = [[dhdt], [djdt], [dmdt], [dddt], [dfCadt], [df1dt], [df2dt], [drdt], [dqdt],
             [dXr1dt], [dXr2dt], [dXsdt], [dXfdt], [dgdt], [dVdt], [dNaidt], [dCaidt], [dCasrdt]]
    
    return deriv             
             

In [7]:
# Last AP extract 

def lastAP(t,V,stimtimes):
    
     if (empty(stimtimes)):
        peakV = max(V) 
        restV = min(V) 
        Vamplitude = peakV - restV 
        a.ravel().nonzero()
        below = V < restV + 0.5*Vamplitude
        above = V > restV + 0.5*Vamplitude
        below_dices = [[below.ravel().nonzero()],[length(t)+1]] 
        above_dices = [[above.ravel().nonzero()],[length(t)+1]] 
            
        intervals1 = np.diff(below_dices) > 5
        intervals1 = intervals1.ravel().nonzero()
        dices_up = below_dices[intervals1]
        times_up = t(below_dices(intervals1))
            
        intervals2 = np.diff(above_dices) > 5
        intervals2 = intervals2.ravel().nonzero()
        dices_down = above_dices(intervals2) 
        times_down = t(above_dices(intervals2))
            
            if (empty(dices_up) or empty(dices_down): 
                keepT = nan*ones(len(t),1) 
                keepV = nan*ones(len(V),1) 

IndentationError: unexpected indent (<ipython-input-7-ca58c188ef13>, line 25)

In [52]:
# iPSC fitness

def iPSC_fitness_func(x):

    tend_fit = max(ventricleTime)
    dt_fit = tend_fit/len(ventricleTime)
    dt_fit = round(dt_fit,1)
    
    minTemp = 310.15
    maxTemp = 310.15
    tempStep = 1
    pace = 0
    defQ10 = 1.0
    
    ###### Model Parameters #######  
    
    # Temp 
    Q10_NaK = 1.63
    Q10_KmNai = 1.49
    Q10_CaL = 1.8
    Q10_NCX = 1.57
    Q10_SLCaP = 2.35
    Q10_SRCaP = 2.6
    Q10_K1 = 1.5
    Q10_Kr = 4.4
    Q10_rel = 1.25    
    Q10_f = 1.38
    Q10_to = 3.6
    Q10_Na = 2.2
    Q10_Ks = 3.5
    Q10_bNa = defQ10
    Q10_bCa = defQ10
    Q10_leak = defQ10    
    
    # Extracellular Conc
    Nao = 151
    Ko = 5.4
    Cao = 1.8
    Ki = 150
    
    # Cell size/dimension
    if (celltype==1):
        Cm = 98.7109e-12
        Vc = 8800
        Vsr = 583.73
    else:
        Cm = 78.6671e-12
        Vc = 7012
        Vsr = 465.20
        
    # Uniform Conduct. and Currents    
    GNa = x[0]
    GK1 = x[1]
    GKr = x[2]
    GKs = x[3]
    Gto = x[4]
    GCaL = x[5]
    KNaCA = x[6]
    PNaK = x[7]
    GpCa = x[8]
    GbNa = x[9]
    GbCa = x[10]
    G_RyR = x[11]
    Vmaxup = x[12]
    Vleak = x[13]
    Gf = x[14]
    arel = 16.464
    brel = 0.25
    crel = 8.232
    
    # Uniform constants
    Bufc = 0.25
    Bufsr = 10
    Kbufc = 0.001
    Kbufsr = 0.3
    Kup = 0.00025
    KpCa = 0.0005
    F = 96485.3415
    R = 8.314472 
    T = 298.15
    L0 = 0.025
    Pkna = 0.03
    Ksat = 0.1
    KmCa = 1.38 
    KmNai = 87.5 
    alpha = 2.8571432 
    gamma = 0.35
    KmNa = 40
    Kmk = 1
    
    # Initial Conditions
    h0 = 0.75
    j0 = 0.75
    m0 = 0
    d0 = 0
    fCa0 = 1
    f1comma0 = 1
    f2comma0 = 1
    r0 = 0
    q0 = 1
    Xr10 = 0
    Xr20 = 1
    Xs0 = 0 
    Xf0 = 0.1
    g0 = 1
    V0 = -70e-3
    
    if (celltype==1):
        Nai0 = 10
    else:
        Nai0 = 14.1

    Cai0 = 0.0002
    Casr0 = 0.3
    


    statevar_i =np.array([[h0],[j0],[m0],[d0],[fCa0],[f1comma0],
                          [f2comma0],[r0],[q0],[Xr10],[Xr20],[Xs0],
                          [Xf0],[g0],[V0],[Nai0],[Cai0],[Casr0]])
    
    globals = [Q10_NaK, Q10_KmNai, Q10_CaL, Q10_NCX, Q10_SLCaP, Q10_SRCaP,
     Q10_f, Q10_K1, Q10_Na, Q10_to, Q10_Kr, Q10_Ks, Q10_bNa, Q10_bCa,
     Q10_leak, Q10_rel, Nao, Ko, Cao, Ki, Cm, Vc, Vsr, arel, brel, crel,
     Bufc, Bufsr, Kbufc, Kbufsr, Kup, KpCa, F, R, T, L0, Pkna, Ksat, KmCa,
     KmNai, alpha, gamma, KmNa, Kmk, GNa, GK1, GKr, GKs, Gto, GCaL, PNaK,
     KNaCA, GpCa, GbNa, GbCa, G_RyR, Vmaxup, Vleak, Gf, celltype]
    
    tend = 10
    secondtokeep = 2
    Istim = 0
    
    #odeint(model,initial condition, tspan,)
    [post,posstatevars] = odeint(dydt_paci13_temp,statevar_i,[0,(tend-secondtokeep)],args=(globalVars,Istim),h0=2e-5,mxstep=1e-3)
    statevar_i = posstatevars[-1,:] #-1 means last element
    
    [t,statevars] = odeint(dydt_paci13_temp,statevar_i,[0,secondtokeep],args=(globalVars,Istim),h0=2e-5,mxstep=1e-3)
    statevar_i = statevars[-1,:]
    
    h = statevars[:,1]
    j = statevars[:,2]
    m = statevars[:,3] 
    d = statevars[:,4] 
    fCa = statevars[:,5] 
    f1comma = statevars[:,6] 
    f2comma = statevars[:,7] 
    r = statevars[:,8] 
    q = statevars[:,9] 
    Xr1 = statevars[:,10] 
    Xr2 = statevars[:,11] 
    Xs = statevars[:,12] 
    Xf = statevars[:,13] 
    g = statevars[:,14] 
    V = statevars[:,15] 
    Nai = statevars[:,16] 
    Cai = statevars[:,17] 
    Casr = statevars[:,18] 
    
    t = t-min(t)
    t = t * 1000
    V = V*1000
    
    [t_regular_AP, V_regular_AP] = lastAP(t,V,[])
    
    t_regular_AP = t_regular_AP - min(t_regular_AP)
    arange(10,0,-3)
    t_regular = arange(tend_fit,0,dt_fit)


    if (V_regular_AP == nan):
        V_regular = 1e20*ones(1,length(t_regular))
    else:
        V_regular = interp1(t_regular_AP,V_regular_AP,t_regular)
        
    #fitness calculation 
    y = abs(sum(log((V_regular - ventricleVolt)**2)))
    
    fitness = y
    
    return fitness
    
    
    

    